## Q2.5 Sum Lists

Given two linked lists, each represent a integer, each node is one digit, stored in reverse order. 

Add the two numbers and return sum as a linked list.

e.g. 
- input: (7->1->6) + (5->9->2), that is 617+295
- output: 2->1->9, that is 912

Input: 
- Linked list lll
- Linked list ll2

Return: 
Linked list sumlist

In [2]:
from myLinkedLists import SinglyLinkedList, Node

""" 
Solution1:

1. Traverse both lists, get two integers and add 
2. Transform result integer to linked list

space O(n)
time O(n)

"""

def sumList(ll1_head, ll2_head):
    if not ll1_head and not ll2_head:
        return None
    
    if not ll1_head:
        return SinglyLinkedList(ll2_head)
    if not ll2_head:
        return SinglyLinkedList(ll1_head)
    
    num1 = sllToNum(ll1_head)
    num2 = sllToNum(ll2_head)
    
    res = numTosll(num1+num2)
    return res

def sllToNum(node):
    num = 0
    # 1st: *(10^0), 2nd: *(10^1)..
    order = 0
    while node:
        num += node.val * (10**order)
        node = node.next
        order += 1
    return num
    
def numTosll(num):
    # create a dumbhead
    head = Node(-1)
    curr = head
    while num > 0:
        digit = num % 10
        
        curr.next = Node(digit)
        curr = curr.next 
        
        num //= 10
    
    # remove dumbhead
    return SinglyLinkedList(head.next)

In [3]:
"""
Solution2:

If input list can be changed, we can do this inplace

1. Use two pointers to traverse both lists at the same time. (head is the units, next is the tens...)
2. Add digits at the same idx and carry over to idx+1 if needed
3. Store result digit to sll1 or sll2

space O(1)
time O(n)
"""
def sumList2(head1, head2):
    # get length of both list
    len1 = getLength(head1)
    len2 = getLength(head2)
    
    # store result in longer list
    node1 = head1 if len1 > len2 else head2
    node2 = head2 if len1 > len2 else head1
    res = node1
    
    carry = 0
    while node1 and node2:
        # calculate digit
        _sum = node1.val + node2.val
        digit = _sum % 10
        
        # update longer list (node1)
        node1.val = digit + carry
        
        # update carry after assign val
        carry = _sum // 10
        
        node1 = node1.next
        node2 = node2.next
        
    
    # Since node1's list is longer than node2's, only need to update carry for node1's list
    while node1 and carry > 0:
        _sum = node1.val + carry
        digit = _sum % 10
        node1.val = digit
        
        carry = _sum // 10
        
        node1 = node1.next
    
    return SinglyLinkedList(res)

def getLength(node):
    length = 0
    while node:
        length += 1
        node = node.next
    return length

## Follow up

Suppose digits are stored in forward order

e.g. (1->2->3) + (4->5) = (1->6->8)

In [4]:
"""
Solution1:
same as original sol1 except the calculation between list and number is different

space O(n)
time O(n)
"""
def sumListRev(head1,head2):
    if not head1 and not head2:
        return None
    if not head1:
        return SinglyLinkedList(head2)
    if not head2:
        return SinglyLinkedList(head1)
    
    num1 = sllToNumRev(head1)
    num2 = sllToNumRev(head2)
    
    res = numTosllRev(num1+num2)
    return res

def sllToNumRev(node):
    digits = []
    while node:
        digits.append(str(node.val))
        node = node.next
    num = int(''.join(digits))
    return num
    
def numTosllRev(num):
    numstr = str(num)
    # dumbhead
    head = Node(-1)
    curr = head
    for char in numstr:
        digit = int(char)
        curr.next = Node(digit)
        curr = curr.next
    
    # remove dumbhead
    return SinglyLinkedList(head.next)

In [13]:
"""
Solution2: CCI

Recursion and carry excess backwards <-

1. get lengths of both lists, l1, l2
2. pad abs(l1-l2) zeroes to shorter list
3. two pointers each for a list, start together
4. recursively add the carry and return carry and head_of_sum_list

space O(n)    recursion stack O(n) and new sum list O(n)
time O(n)

"""

def sumListRev2(head1,head2):
    if not head1 and not head2:
        return None
    if not head1:
        return SinglyLinkedList(head2)
    if not head2:
        return SinglyLinkedList(head1)
    
    l1 = getLength(head1)
    l2 = getLength(head2)
    
    if l1 > l2:
        # pad l2
        head2 = padBefore(head2, abs(l1-l2))
    else:
        # pad l1
        head1 = padBefore(head1, abs(l1-l2))
    
    # recursively add
    sumHead, carry = sumListsHelper(head1, head2)
    
    if carry > 0:
        sumHead = insertBefore(sumHead, carry)
    return SinglyLinkedList(sumHead)
        
def padBefore(head, x):
    # pad x zeroes before head
    for i in range(x):
        head = insertBefore(head, 0)
    return head

def insertBefore(head, x):
    # insert a node with val x before head
    node = Node(x)
    node.next = head
    return node
    

def sumListsHelper(head1, head2):
    if not head1 and not head2:
        return None, 0
    # Note: no need to consider head1 or head2 because lengths are padded to the same
    
    sumHead, carry = sumListsHelper(head1.next, head2.next)
    
    val = head1.val + head2.val + carry
    carry = val // 10
    digit = val % 10
    
    sumHead = insertBefore(sumHead, digit)
    return sumHead, carry

If Doubly linked list, we can do:

1. get tails of both list
2. traverse from back to front using prev pointers
3. calculate digit and carry and do this inplace

space O(1)
time O(n)

## Testing 

In [6]:
from myLinkedLists import SinglyLinkedList, Node

def test(test_lists, func):
    total = len(test_lists)
    correct = 0
    
    for l1,l2,sol in test_lists:
        sll1 = SinglyLinkedList(l1)
        sll2 = SinglyLinkedList(l2)
        
        print('Sll1: ', sll1, '\nSll2: ', sll2, '\nsol: ', sol)
        res = func(sll1.head, sll2.head)
        print('res: ', res)
        
        curr = res.toList() == sol
        correct += curr
        print(curr, '\n', '-'*50, sep='')
    
    print(f'{correct}/{total}')
    if correct == total:
        print('All passed')

In [7]:
# (list1, list2, result)
test_lists = [
    ([7,1,6], [5,9,2], [2,1,9]),
    ([2,3], [4,5,6], [6,8,6]),
    ([], [1,2,3], [1,2,3])
]

In [8]:
test(test_lists, sumList2)

Sll1:  7 -> 1 -> 6 
Sll2:  5 -> 9 -> 2 
sol:  [2, 1, 9]
res:  2 -> 1 -> 9
True
--------------------------------------------------
Sll1:  2 -> 3 
Sll2:  4 -> 5 -> 6 
sol:  [6, 8, 6]
res:  6 -> 8 -> 6
True
--------------------------------------------------
Sll1:  Error: This is an empty list. 
Sll2:  1 -> 2 -> 3 
sol:  [1, 2, 3]
res:  1 -> 2 -> 3
True
--------------------------------------------------
3/3
All passed


## Testing Followup

In [9]:
# (list1, list2, result)
test_lists = [
    ([7,1,6], [5,9,2], [1,3,0,8]),
    ([6,1,7], [2,9,5], [9,1,2]),
    ([2,3], [4,5,6], [4,7,9]),
    ([], [1,2,3], [1,2,3])
]

In [14]:
test(test_lists, sumListRev2)

Sll1:  7 -> 1 -> 6 
Sll2:  5 -> 9 -> 2 
sol:  [1, 3, 0, 8]
res:  1 -> 3 -> 0 -> 8
True
--------------------------------------------------
Sll1:  6 -> 1 -> 7 
Sll2:  2 -> 9 -> 5 
sol:  [9, 1, 2]
res:  9 -> 1 -> 2
True
--------------------------------------------------
Sll1:  2 -> 3 
Sll2:  4 -> 5 -> 6 
sol:  [4, 7, 9]
res:  4 -> 7 -> 9
True
--------------------------------------------------
Sll1:  Error: This is an empty list. 
Sll2:  1 -> 2 -> 3 
sol:  [1, 2, 3]
res:  1 -> 2 -> 3
True
--------------------------------------------------
4/4
All passed
